In [1]:
import json
import yaml
import sys
import time
import IPython.display as ipd
import pprint
from pathlib import Path
from tqdm import tqdm

import numpy as np
import torch
import torchaudio
from librosa.filters import mel as librosa_mel_fn
#import matplotlib
#matplotlib.use("Agg")
import matplotlib.pyplot as plt
from scipy.io.wavfile import write


import toybox

## [seq] device setting

In [2]:
import os

print(f"all cpu at using device: {os.cpu_count()}")
print(f"Number of available CPU: {len(os.sched_getaffinity(0))}") # Number of available CPUs can also be obtained. ,use systemcall at linux.

all cpu at using device: 52
Number of available CPU: 4


In [3]:
DEVICE = 'cpu' # 'cuda' or 'cpu'
DEVICE_UTMOS = 'cuda'
if torch.cuda.is_available():
    print('use cuda')
else:
    #os._exit(os.EX_OK)
    print('use cpu')

device = torch.device(DEVICE_UTMOS)
print(f'device: {device}')

use cpu
device: cuda


In [4]:
# seed val from https://github.com/sarulab-speech/UTMOS22/blob/master/strong/configs/train/default.yaml
utmos_seed = 1234
toybox.set_seed(utmos_seed)

## [seq] load model

In [5]:
predictor_utmosv1 = torch.hub.load("tarepan/SpeechMOS:v1.2.0", "utmos22_strong", trust_repo=True)

Using cache found in /work/sora-sa/.cache/torch/hub/tarepan_SpeechMOS_v1.2.0
/work/sora-sa/aoi/.venv/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


## [seq] setting path

In [43]:
model_type = 'stoch' #'deter' or 'stoch'
results_dir_path = Path(f'./result4nixeval/{model_type}')
wav_dir_path = results_dir_path / 'nix_wavs'
eval_json_path = results_dir_path / f'eval4nix_{model_type}.json'


if wav_dir_path.exists()==True:
    print(f'Already Exists {wav_dir_path}')
else:
    print(f'No  {wav_dir_path}')    

if results_dir_path.exists()==True:
    print(f'Already Exists {results_dir_path}')
else:
    print(f'No {results_dir_path}')

if eval_json_path.exists()==True:
    with open(eval_json_path) as f:
        info_jsonl = [json.loads(l) for l in f.readlines()]
else:
    print('No {eval_json_path}')

Already Exists result4nixeval/stoch/nix_wavs
Already Exists result4nixeval/stoch


## test_run

In [8]:
wav_path = wav_dir_path / f'{info_jsonl[0]["name"]}.wav'
print(wav_path)
waveform, samplerate = torchaudio.load(wav_path)
utmosv1_score = predictor_utmosv1(waveform, samplerate)
print(f'utmosv1_score: {utmosv1_score.item()}')
ipd.display(ipd.Audio(waveform, rate=samplerate))

result4nixeval/stoch/nix_wavs/LJ045-0049.wav
utmosv1_score: 3.7527384757995605


## [seq] eval

In [9]:
infer_data_num: int = 101 #len(test_ds_list) is 200
print(infer_data_num)
print(wav_dir_path)
print(info_jsonl[0])

101
result4nixeval/stoch/nix_wavs
{'name': 'LJ045-0049', 'path': 'result4eval/stoch/nix_wavs/LJ045-0049.wav', 'text': 'Lee would say that it would be better for me if I went to Russia. I did not know why. I did not know what he had in mind.', 'audio_playback_time': 7.209795918367347, 'dt': 0.9196576047688723, 'RTF4wav': 0.12755667638608112}


In [10]:
eval_list = []

for index in tqdm(range(infer_data_num)):
    filename = info_jsonl[index]["name"]
    wav_path = wav_dir_path / f'{filename}.wav'
    print(wav_path)
    waveform, samplerate = torchaudio.load(wav_path)
    utmosv1_score = predictor_utmosv1(waveform, samplerate)
    utmosv1_score_float = utmosv1_score.item()
    print(f'utmosv1: {utmosv1_score_float}')
    eval_dict = {
        'name': filename,
        'path': str(wav_path),
        'utmos': utmosv1_score_float
    }
    eval_list.append(eval_dict)


eval_json_path = results_dir_path / f'eval4nix_utmosv1_{model_type}.json'
print(f'eval_json_path: {eval_json_path}')
if eval_json_path.exists() == False:
    with open(eval_json_path, 'w') as f:
        for entry in eval_list:
            f.write(json.dumps(entry) + '\n')
    print(f'Make {eval_json_path}')
else:
    print(f'Already Exists {eval_json_path}')
print('fin')

  0%|                                                                                                     | 0/101 [00:00<?, ?it/s]

result4nixeval/stoch/nix_wavs/LJ045-0049.wav


  2%|█▊                                                                                           | 2/101 [00:00<00:32,  3.07it/s]

utmosv1: 3.7527384757995605
result4nixeval/stoch/nix_wavs/LJ017-0027.wav
utmosv1: 4.30259370803833
result4nixeval/stoch/nix_wavs/LJ023-0031.wav


  3%|██▊                                                                                          | 3/101 [00:00<00:28,  3.43it/s]

...

utmosv1: 3.7303009033203125
result4nixeval/stoch/nix_wavs/LJ029-0081.wav


 99%|██████████████████████████████████████████████████████████████████████████████████████████ | 100/101 [00:35<00:00,  2.01it/s]

utmosv1: 4.068802833557129
result4nixeval/stoch/nix_wavs/LJ017-0230.wav


100%|███████████████████████████████████████████████████████████████████████████████████████████| 101/101 [00:36<00:00,  2.78it/s]

utmosv1: 3.8914380073547363
eval_json_path: result4nixeval/stoch/eval4nix_utmosv1_stoch.json
Make result4nixeval/stoch/eval4nix_utmosv1_stoch.json
fin


## recheck

In [44]:
eval_jsonl_path = results_dir_path / f'eval4nix_utmosv1_{model_type}.json'
print(f'eval_jsonl_path: {eval_jsonl_path}')

if eval_jsonl_path.exists() == True:
    print(f'Exist {eval_jsonl_path}')
    import json
    with open(eval_jsonl_path) as f:
        eval_jsonl_list = [json.loads(l) for l in f]
else:
    print(f'No Exists {eval_jsonl_path}')

eval_jsonl_path: result4nixeval/stoch/eval4nix_utmosv1_stoch.json
Exist result4nixeval/stoch/eval4nix_utmosv1_stoch.json


In [45]:
diff_result = list(filter(lambda x: x not in eval_list, eval_jsonl_list))
print(diff_result)

[]


## check and culc Indicator

In [17]:
model_type = 'stoch' #'deter' or 'stoch'
results_dir_path = Path(f'./result4nixeval/{model_type}')
wav_dir_path = results_dir_path / 'nix_wavs'
eval_json_path = results_dir_path / f'eval4nix_utmosv1_{model_type}.json'
print(eval_json_path)

if wav_dir_path.exists()==True:
    print(f'Already Exists {wav_dir_path}')
else:
    print(f'No  {wav_dir_path}')    

if results_dir_path.exists()==True:
    print(f'Already Exists {results_dir_path}')
else:
    print(f'No {results_dir_path}')

if eval_json_path.exists()==True:
    with open(eval_json_path) as f:
        eval_utmosv1_json = [json.loads(l) for l in f.readlines()]
else:
    print('No {eval_json_path}')

result4nixeval/stoch/eval4nix_utmosv1_stoch.json
Already Exists result4nixeval/stoch/nix_wavs
Already Exists result4nixeval/stoch


In [18]:
#diff_result = list(filter(lambda x: x not in eval_jsonl_list, info_jsonl1))
#print(diff_result)

In [19]:
def round_significant_digits(value, significant_digits=5):
    if value == 0:
        return 0

    import math
    scale = math.floor(-math.log10(abs(value)))  # Find the first nonzero after the decimal point
    factor = 10 ** (scale + significant_digits - 1)  # Scale to hold 5 significant digits

    rounded_value = round(value * factor,1) / factor  # Adjust and round off the scale
    return rounded_value

In [20]:
significant_digits = 5

utmos_list = [eval_utmosv1_json[n]['utmos'] for n in range(len(eval_utmosv1_json))]
utmos_nparr = np.array(utmos_list[1:101])
print(utmos_nparr.shape)
utmos_mean = round_significant_digits(np.mean(utmos_nparr), significant_digits=significant_digits)
utmos_var = round_significant_digits(np.var(utmos_nparr), significant_digits=significant_digits)
utmos_std = round_significant_digits(np.std(utmos_nparr), significant_digits=significant_digits)
print(f'utmos mean: {utmos_mean}')
print(f'utmos var: {utmos_var}')
print(f'utmos std: {utmos_std}')

(100,)
utmos mean: 4.0298
utmos var: 0.082376
utmos std: 0.28701


In [21]:
"""
utmos mean: 4.0298
utmos var: 0.082376
utmos std: 0.28701
"""

'\nutmos mean: 4.0298\nutmos var: 0.082376\nutmos std: 0.28701\n'